**Sustainable Software Development, block course, March 2021**  
*Scientific Software Center, Institute for Scientific Computing, Dr. Inga Ulusoy*

# Analysis of the data

Imagine you perform a "measurement" of some type and obtain "scientific data". You know what your data represents, but you have only a vague idea how different features in the data are connected, and what information you can extract from the data.

You would start first with going through the data, making sure your data set is complete and that the result is reasonable. Imagine this already happened.

In the next step, you would inspect your data more closely and try to identify structures. That is the step that we are focusing on in this unit.

In the `data` folder, you will find several data files (`*.t` and `*.dat`). These are data files generated through some "new approach" that hasn't been used in your lab before. No previous analysis software exists, and you are going to establish a protocol for this "new approach" and "publish your results".

The data can be grouped into two categories: 
1. data to be analyzed using statistical methods;
2. data to be analyzed using numerical methods.

In your hypothetical lab, you are an "expert" in one particular "method", and your co-worker is an "expert" in the other. Combined these two methods will lead to much more impactful results than if only one of you analyzed the data. Now, the task in this course is to be solved collaboratively with your team member working on one of the analysis approaches, and you working on the other. You will both implement functionality into the same piece of "software", but do so collaboratively through git.

As you do not know yet which analysis is most meaningful for your data, and how to implement it, you will start with a jupyter notebook. You and your team member will work on the same notebook that will be part of a github repository for your project. This is the task for today. Discuss with your team members who will work on the statistical and who on the numerical analysis.

## Step 1

Accept the assignment through GitHub classroom.

## Step 2

Clone the repository to your local machine.

## Step 3

Start working on task 1 for your analysis approach. 

## Step 4

Create your own branch of the repository and commit your changes to your branch; push to the remote repository.

## Step 5

Open a `pull request` so your team member can review your implementation. Likewise, your team member will ask you to review theirs.

## Step 6

Merge the changes in your branch into `main`. Resolve conflicts.

## Step 7

Repeat working on task; committing and pushing to your previously generated branch or a new branch; open a pull request; merge with main; until you have finished all the tasks in your analysis approach. Delete obsolete branches.

# Start of the analysis notebook

**Author : Lynn Buchele**  
*Date : 22.01.2024*  
*Affiliation : The entity under whose name you are working on this*  

Place the required modules in the top, followed by required constants and global functions.

In [ ]:
# Required modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np 

In [ ]:
# Constants and global functions

def drop_low_variance(data_frame, var_threshold):
    """Drop columns from the data frame with variance below a threshold."""
    
    # Get the columns to drop
    columns_to_drop = [col for col in data_frame.columns if data_frame.var()[col] < var_threshold]

    # Print a message about the columns to drop
    print('These columns will be discarded as their variance is below the set threshold', columns_to_drop)

    # Drop the columns from the data frame
    data_frame.drop(columns=columns_to_drop, inplace=True)

    return data_frame

In [ ]:
# Reading of the data files

# Statistical analysis

Find correlations in the data sets. Analyse the data statistically and plot your results.  

Here we would want to do everything with pandas and leave the data in a dataframe. The files that are relevant to you are `expect.t`, `npop.t` and `table.dat`.

### Task 1: Read in expec.t and plot relevant data

In [ ]:
# Read the expec.t file
expect_dat = pd.read_csv('../data/expec.t', sep='\s+') #noqa

In [ ]:
# Plot the file content
expect_dat

We can discard the entries norm, \<x>, and \<y> as these are mostly constant.

In [ ]:
# Eliminate columns based on the variance: if the variance of the values
# in a column is below a given threshold, that column is discarded
expect_dat = drop_low_variance(expect_dat, 1e-5)  # var threshold taken from inspecting data

### Task 2: Create plots of the relevant data and save as .pdf.

In [ ]:
# Open a figure with specific size
plt.figure(figsize=[40, 40])

# Create a pairplot
seaborn.pairplot(expect_dat)  # Use pairplot since we're looking for correlations

# Save the plot as a .pdf
plt.savefig('../results/expect_data_pairplot.pdf', format='pdf')

### Task 3: Read in file `npop.t` and analyze correlations in the data

In [ ]:
# Read the npop.t file
npop_dat = pd.read_csv('../data/npop.t', sep='\s+') #noqa

In [ ]:
# Discard all columns with variance below a set threshold - we can consider them as constant
npop_dat = drop_low_variance(npop_dat, 1e-5)  # keep var threshold from previous data

In [ ]:
# Plot the file content
npop_dat

Plot the remaining columns. Seaborn prefers "long format" (one column for all measurement values, one column to indicate the type) as input, whereas the cvs is in "wide format" (one column per measurement type).

In [ ]:
# Open a figure
plt.figure(figsize=[40, 40])

# Create a pairplot
seaborn.pairplot(npop_dat)

## Quantify the pairwise correlation in the data

- negative correlation: y values decrease for increasing x - large values of one feature correspond to small values of the other feature
- weak or no correlation: no trend observable, association between two features is hardly observable
- positive correlation: y values increase for decreasing x - small values of one feature correspond to small values of the other feature

Remember that correlation does not indicate causation - the reason that two features are associated can lie in their dependence on same factors.

Correlate the value pairs using Pearson's $r$. Pearson's $r$ is a measure of the linear relationship between features:

$r = \frac{\sum_i(x_i − \bar{x})(y_i − \bar{y})}{\sqrt{\sum_i(x_i − \bar{x})^2 \sum_i(y_i − \bar{y})^2}}$

Here, $\bar{x}$ and $\bar{y}$ indicate mean values. $i$ runs over the whole data set. For a positive correlation, $r$ is positive, and negative for a negative correlation, with minimum and maximum values of -1 and 1, indicating a perfectly linear relationship. Weakly or not correlated features are characterized by $r$-values close to 0.

Other measures of correlation that can be used are Spearman's rank (value pairs follow monotonic function) or Kendall's $\tau$ (measures ordinal association), but they do not apply here. You can also define measures yourself.

In [ ]:
# Print the correlation matrix
npop_dat.corr(method='pearson')

The diagonal values tell us that each value is perfectly correlated with itself. We are not interested in the diagonal values and also not in the correlation with time. We also need to get rid of redundant entries. Finally, we need to find the value pairs that exhibit the highest linear correlation. We still want to know if it is positive or negative correlation, so we cannot get rid of the sign.

In [ ]:
# Calculate the correlation matrix without the time column
corr_matrix = npop_dat.drop(columns=['time']).corr(method='pearson')

# Set the diagonal elements of the correlation matrix to None
for i in range(corr_matrix.shape[0]):
    corr_matrix.iloc[i, i] = None

# Stack the correlation matrix, remove duplicates and reset the index
corr_matrix = corr_matrix.stack().reset_index()


In [ ]:
# Get rid of the time column, lower triangular and diagonal entries of the correlation matrix
# Sort the remaing values according to their absolute value, but keep the sign

# Calculate the correlation matrix without the time column
corr_matrix = npop_dat.drop(columns=['time']).corr(method='pearson')

# Set the diagonal elements of the correlation matrix to None
for i in range(corr_matrix.shape[0]):
    corr_matrix.iloc[i, i] = None

# Stack the correlation matrix into a table, remove duplicates and reset the index
corr_table = corr_matrix.stack().reset_index()

# Set the column names for the table
corr_table.columns = ['var1', 'var2', 'corr']

# Sort the table by the absolute correlation values, reset the index for readability of display
corr_table = corr_table.sort_values(by='corr', ascending=False, key=abs).reset_index(drop=True)

# Print the correlation table
corr_table

Note that the entries in the left column are not repeated if they do not change from the row above (so the fourth feature pair is MO3 and MO6).

### Task 4: Print the resulting data to a file

In [ ]:
# Print the correlation table to a file
corr_table.to_csv('../results/npop_correlations.dat', sep='\t', index=False)

### Task 5: Calculate the Euclidean distance (L2 norm) for the vectors in `table.dat`


The Euclidean distance measures the distance between to objects that are not points:

$d(p,q) = \sqrt{\left(p-q\right)^2}$

In this case, consider each of the columns in table.dat as a vector in Euclidean space, where column $r(x)$ and column $v(x)$ denote a pair of vectors that should be compared, as well as $r(y)$ and $v(y)$, and r(z) and v(z).

(Background: These are dipole moment components in different gauges, the length and velocity gauge.)

In [ ]:
# read in table.dat - I suggest reading it as a numpy array
# replace the NaNs by zero
table_dat = np.genfromtxt('../data/table.dat', skip_header=1)
table_dat = np.nan_to_num(table_dat)



Now calculate how different the vectors in column 2 are from column 3, column 4 from column 5, and column 6 from column 7.

In [ ]:
# calculate the Euclidean distance
def Euclidean_distance(v1, v2):
    distance = np.sqrt(np.sum((v1-v2)**2))
    return distance

# Split of table_dat accoding to names and calculate corresponding distances
rx = table_dat[2]
vx = table_dat[3]
x_distance = Euclidean_distance(rx, vx) 

ry = table_dat[4]
vy = table_dat[5]
y_distance = Euclidean_distance(ry, vy) 

rz = table_dat[6]
vz = table_dat[7]
z_distance = Euclidean_distance(rz, vz) 


In [ ]:
# plot the result and save to a .pdf
plt.figure()
plt.bar(np.arange(0,3),np.array([x_distance, y_distance, z_distance]), tick_label=['x','y','z'] )
plt.title('Distances Between u and v')
plt.savefig('../results/table_dat_distances.pdf', format='pdf')

In [ ]:
# print the result to a file

# Numerical analysis

Analyze the data using autocorrelation functions and discrete Fourier transforms. Plot your results.

In [ ]:
# define some global functions

### Task 1: Read in `efield.t` and Fourier-transform relevant columns

In [ ]:
# Read the efield.t file
efield_dat = pd.read_csv('../data/efield.t', sep='\s+') #noqa

In [ ]:
# Plot the file content
efield_dat

Here we are interested in column 2 since the others are constant.

In [ ]:
# Discard the columns with variance below threshold - these are considered constant
efield_dat = drop_low_variance(efield_dat, 1e-5)

In [ ]:
# Discrete Fourier transform of the remaining column: You only need the real frequencies
sp = np.fft.fft(efield_dat.loc[:, 'y'])
freq = np.fft.fftfreq(efield_dat.loc[:, 'y'].shape[-1])

### Task 2: Generate a plot of your results to be saved as pdf.

In [ ]:
# Open a figure
plt.figure(figsize=(20, 10))

# Create a line plot with the real frequencies
plt.plot(freq, sp.real)

# Plot styling
plt.title('Fourier transform of efield data', fontsize=18)
plt.xlabel('Frequency', fontsize=12)

# Save the plot to a .pdf
plt.savefig('../results/efield_fourier_transform.pdf', format='pdf')

### Task 3: Calculate the autocorrelation function from nstate_i.t
The autocorrelation function measures how correlated subsequent vectors are with an initial vector; ie. 

$\Psi_{corr} = \langle \Psi(t=0) | \Psi(t) \rangle = \int_0^{tfin} \Psi(0)^* \Psi(t) dt$

Since we are in a numerical representation, the integral can be replaced with a sum; and the given vectors are already normalized.

In [ ]:
# read in as numpy array
nstate_dat = np.genfromtxt('../data/nstate_i.t', skip_header=1)
nstate_dat = np.nan_to_num(nstate_dat)

In [ ]:
# store the time column (column 0) in a vector and drop from array
time = nstate_dat[:, 0]
nstate_dat = nstate_dat[:, 1:]

In [ ]:
# correct the data representation: this is in fact a complex matrix
# the real part of each matrix column is contained in numpy array column 0, 2, 4, 6, ...
# the imaginary part of each matrix column is contained in numpy array column 1, 3, 5, 7, ...
# convert the array that was read as dtype=float into a dtype=complex array

In [ ]:
result = np.empty((nstate_dat.shape[0], int(nstate_dat.shape[1]/2)), dtype=complex)

In [ ]:
result.real = nstate_dat[:, ::2]

In [ ]:
result.imag = nstate_dat[:, 1::2]

In [ ]:
# for the autocorrelation function, we want the overlap between the first vector at time 0 and all
# subsequent vectors at later times - the sum of the product of initial and subsequent vectors for each time step
sum = 0
for i in range(result[:, ].shape[0]):
    temp = np.dot(result[:, 0], result[:, i])
    sum+=temp

### Task 4: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the autocorrelation function - real, imaginary and absolute part
plt.bar([1, 2, 3], [sum.real, sum.imag, np.absolute(sum)])
plt.show()

### Task 5: Discrete Fourier transform of the autocorrelation function

In [ ]:
# discrete Fourier-transform the autocorrelation function - now we need all frequency components,
# also the negative ones


### Task 6: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the power spectrum (abs**2)